In [100]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()


In [101]:
# Question 1
spark.version

'3.5.2'

In [102]:
# Question 2
spark.read \
    .parquet('yellow_tripdata_2024-10.parquet') \
    .repartition(4) \
    .write \
    .parquet('./trips')

In [103]:
# Question 3
from pyspark.sql.functions import date_format, col

df = spark.read.parquet('yellow_tripdata_2024-10.parquet')
df = df.withColumn('pickup_date', date_format('tpep_pickup_datetime', 'yyyy-MM-dd')) \
    .withColumn('dropoff_date', date_format('tpep_dropoff_datetime', 'yyyy-MM-dd')) \
    .filter(col('pickup_date') == '2024-10-15') \
    .filter(col('dropoff_date') == '2024-10-15') \
    .filter(col('trip_distance') > 0)

df.count()

125212

In [104]:
# Question 4
from pyspark.sql.functions import unix_timestamp

df = spark.read.parquet('yellow_tripdata_2024-10.parquet')
df = df.withColumn('hour_durations', (unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 3600) \
        .orderBy('hour_durations', ascending=False)
df.select('hour_durations').show(1)

+------------------+
|    hour_durations|
+------------------+
|162.61777777777777|
+------------------+
only showing top 1 row



In [105]:
# Question 6
from pyspark.sql.functions import count

df = spark.read.parquet('yellow_tripdata_2024-10.parquet')

res_id = df.groupBy(df.PULocationID) \
    .agg(count(df.PULocationID).alias('count')) \
    .orderBy('count') \
    .select('PULocationID') \
    .first()['PULocationID']

spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv('taxi_zone_lookup.csv') \
    .filter(col('LocationID') == res_id) \
    .show(truncate=False)

+----------+---------+---------------------------------------------+------------+
|LocationID|Borough  |Zone                                         |service_zone|
+----------+---------+---------------------------------------------+------------+
|105       |Manhattan|Governor's Island/Ellis Island/Liberty Island|Yellow Zone |
+----------+---------+---------------------------------------------+------------+

